[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/examples-gallery/blob/main/development/tabular-classification/documentation-tutorial/tabular-tutorial-part-3.ipynb)

# <a id="top">Openlayer tabular tutorial - Part 3</a>

Welcome! This is the third notebook from the tabular tutorial. Here, we solve the **data consistency** issues and commit the new datasets and model versions to the platform. You should use this notebook together with the **tabular tutorial from our documentation**.


## <a id="toc">Table of contents</a>

1. [**Fixing the data consistency issues and re-training the model**](#1)
    

2. [**Using Openlayer's Python API**](#2)

In [ ]:
%%bash

if [ ! -e "requirements.txt" ]; then
    curl "https://raw.githubusercontent.com/openlayer-ai/examples-gallery/main/development/tabular-classification/documentation-tutorial/requirements.txt" --output "requirements.txt"
fi

In [ ]:
!pip install -r requirements.txt

## <a id="1"> 1. Fixing the data integrity issues and re-training the model </a>

[Back to top](#top)

In this first part, we will download the data with the consistency issues fixed. This includes dropping rows from the training set that were present in the validation set, as identified in the tutorial.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

### <a id="download">Downloading the dataset </a>

We have stored the dataset on the following S3 bucket. If, for some reason, you get an error reading the csv directly from it, feel free to copy and paste the URL in your browser and download the csv file. The dataset we use is a modified version of the Churn Modeling dataset from [this Kaggle competition](https://www.kaggle.com/competitions/churn-modelling/overview).

In [ ]:
%%bash

if [ ! -e "churn_train_consistency_fix.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/documentation/churn_train_consistency_fix.csv" --output "churn_train_consistency_fix.csv"
fi

if [ ! -e "churn_val_consistency_fix.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/documentation/churn_val_consistency_fix.csv" --output "churn_val_consistency_fix.csv"
fi

In [ ]:
train_df = pd.read_csv("./churn_train_consistency_fix.csv")
val_df = pd.read_csv("./churn_val_consistency_fix.csv")

In [ ]:
feature_names = [
    "CreditScore", 
    "Geography",
    "Gender",
    "Age", 
    "Tenure",
    "Balance",
    "NumOfProducts",
    "HasCrCard",
    "IsActiveMember",
    "EstimatedSalary"
]
label_column_name = "Exited"

x_train = train_df[feature_names]
y_train = train_df[label_column_name]

x_val = val_df[feature_names]
y_val = val_df[label_column_name]

### <a id="prepare">Preparing the data</a>

In [ ]:
def data_encode_one_hot(df, encoders):
    """ Encodes categorical features using one-hot encoding. """
    df = df.copy(True)
    df.reset_index(drop=True, inplace=True) # Causes NaNs otherwise
    for feature, enc in encoders.items():
        enc_df = pd.DataFrame(enc.transform(df[[feature]]).toarray(), columns=enc.get_feature_names_out([feature]))
        df = df.join(enc_df)
        df = df.drop(columns=feature)
    return df

In [ ]:
def create_encoder_dict(df, categorical_feature_names):
    """ Creates encoders for each of the categorical features. 
        The predict function will need these encoders. 
    """
    from sklearn.preprocessing import OneHotEncoder
    encoders = {}
    for feature in categorical_feature_names:
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit(df[[feature]])
        encoders[feature] = enc
    return encoders

In [ ]:
encoders = create_encoder_dict(x_train, ['Geography', 'Gender'])

In [ ]:
x_train_one_hot = data_encode_one_hot(x_train, encoders)
x_val_one_hot = data_encode_one_hot(x_val, encoders)

### <a id="train">Training the model</a>

In [ ]:
sklearn_model = GradientBoostingClassifier(random_state=1300)
sklearn_model.fit(x_train_one_hot, y_train)

In [ ]:
print(classification_report(y_val, sklearn_model.predict(x_val_one_hot)))

## <a id="2"> 2. Using Openlayer's Python API</a>

[Back to top](#top)

Now it's time to upload the datasets and model to the Openlayer platform.

In [ ]:
!pip install openlayer

### <a id="client">Instantiating the client</a>

In [ ]:
import openlayer

client = openlayer.OpenlayerClient("YOUR_API_KEY_HERE")

### <a id="project">Creating a project on the platform</a>

In [ ]:
from openlayer.tasks import TaskType

project = client.create_or_load_project(
    name="Churn Prediction",
    task_type=TaskType.TabularClassification,
    description="Evaluation of ML approaches to predict churn"
)

### <a id="dataset">Uploading datasets</a>

The datasets haven't changed much from the previous version to this one. Thus, the config are essentially the same.

As usual, let's start by augmenting the datasets with the extra columns:

In [ ]:
# Adding the column with the labels
training_set = x_train.copy(deep=True)
training_set["Exited"] = y_train.values
validation_set = x_val.copy(deep=True)
validation_set["Exited"] = y_val.values

In [ ]:
# Adding the column with the predictions (since we'll also upload a model later)
training_set["predictions"] = sklearn_model.predict_proba(x_train_one_hot).tolist()
validation_set["predictions"] = sklearn_model.predict_proba(x_val_one_hot).tolist()

Now, we can prepare the configs for the training and validation sets.

In [ ]:
# Some variables that will go into the `dataset_config`
categorical_feature_names = ["Gender", "Geography"]
class_names = ["Retained", "Exited"]
feature_names = list(x_val.columns)
label_column_name = "Exited"
prediction_scores_column_name = "predictions"

In [ ]:
# Note the camelCase for the dict's keys
training_dataset_config = {
    "categoricalFeatureNames": categorical_feature_names,
    "classNames": class_names,
    "featureNames":feature_names,
    "label": "training",
    "labelColumnName": label_column_name,
    "predictionScoresColumnName": prediction_scores_column_name,
}

In [ ]:
import copy

validation_dataset_config = copy.deepcopy(training_dataset_config)

# In our case, the only field that changes is the `label`, from "training" -> "validation"
validation_dataset_config["label"] = "validation"

In [ ]:
# Training set
project.add_dataframe(
    dataset_df=training_set,
    dataset_config=training_dataset_config
)

In [ ]:
# Validation set
project.add_dataframe(
    dataset_df=validation_set,
    dataset_config=validation_dataset_config
)

We can check that both datasets are now staged using the `project.status()` method. 

In [ ]:
project.status()

### <a id="model">Uploading models</a>

Once we're done with the consistency tests, we'll move on to performance tests, which have to do with the model itself. Therefore, now, we will upload a **full model** instead of a shell model. We will do so so that we can have explain the model's predictions on the platform using explainability techiques such as LIME and SHAP.

#### <a id="full-model"> Full models </a>

To upload a full model to Openlayer, you will need to create a **model package**, which is nothing more than a folder with all the necessary information to run inference with the model. The package should include the following:
1. A `requirements.txt` file listing the dependencies for the model.
2. Serialized model files, such as model weights, encoders, etc., in a format specific to the framework used for training (e.g. `.pkl` for sklearn, `.pb` for TensorFlow, and so on.)
3. A `prediction_interface.py` file that acts as a wrapper for the model and implements the `predict_proba` function. 

Other than the model package, a `model_config.yaml` file is needed, with information about the model to the Openlayer platform, such as the framework used, feature names, and categorical feature names.

Lets prepare the model package one piece at a time.

In [ ]:
# Creating the model package folder (we'll call it `model_package`)
!mkdir model_package

**1. Adding the `requirements.txt` to the model package**

In [ ]:
!scp requirements.txt model_package

**2. Serializing the model and other objects needed**

In [ ]:
import pickle 

# Trained model
with open("model_package/model.pkl", "wb") as handle:
    pickle.dump(sklearn_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Encoder for the categorical features
with open("model_package/encoders.pkl", "wb") as handle:
    pickle.dump(encoders, handle, protocol=pickle.HIGHEST_PROTOCOL)

**3. Writing the `prediction_interface.py` file**

In [ ]:
%%writefile model_package/prediction_interface.py

import pickle
from pathlib import Path

import pandas as pd
from sklearn.preprocessing import OneHotEncoder

PACKAGE_PATH = Path(__file__).parent


class SklearnModel:
    def __init__(self):
        """This is where the serialized objects needed should
        be loaded as class attributes."""

        with open(PACKAGE_PATH / "model.pkl", "rb") as model_file:
            self.model = pickle.load(model_file)
        with open(PACKAGE_PATH / "encoders.pkl", "rb") as encoders_file:
            self.encoders = pickle.load(encoders_file)

    def _data_encode_one_hot(self, df: pd.DataFrame) -> pd.DataFrame:
        """Pre-processing needed for our particular use case."""

        df = df.copy(True)
        df.reset_index(drop=True, inplace=True)  # Causes NaNs otherwise
        for feature, enc in self.encoders.items():
            enc_df = pd.DataFrame(
                enc.transform(df[[feature]]).toarray(),
                columns=enc.get_feature_names_out([feature]),
            )
            df = df.join(enc_df)
            df = df.drop(columns=feature)
        return df

    def predict_proba(self, input_data_df: pd.DataFrame):
        """Makes predictions with the model. Returns the class probabilities."""

        encoded_df = self._data_encode_one_hot(input_data_df)
        return self.model.predict_proba(encoded_df)


def load_model():
    """Function that returns the wrapped model object."""
    return SklearnModel()

**Creating the `model_config.yaml`**

In [ ]:
import yaml

model_config = {
    "name": "Churn classifier",
    "architectureType": "sklearn",
    "metadata": {  # Can add anything here, as long as it is a dict
        "model_type": "Gradient Boosting Classifier",
        "regularization": "None",
        "encoder_used": "One Hot",
    },
    "classNames": class_names,
    "featureNames": feature_names,
    "categoricalFeatureNames": categorical_feature_names,
}

with open("model_config.yaml", "w") as model_config_file:
    yaml.dump(model_config, model_config_file, default_flow_style=False)

Lets check that the model package contains everything needed:

In [ ]:
from openlayer.validators import model_validators

model_validator = model_validators.get_validator(
    task_type=TaskType.TabularClassification,
    model_package_dir="model_package", 
    model_config_file_path="model_config.yaml",
    sample_data = x_val.iloc[:10, :],
)
model_validator.validate()

All validations are passing, so we are ready to add the full model!

In [ ]:
project.add_model(
    model_package_dir="model_package",
    model_config_file_path="model_config.yaml",
    sample_data=x_val.iloc[:10, :],
)

We can check that both datasets and model are staged using the `project.status()` method.

In [ ]:
project.status()

### <a id="commit"> Committing and pushing to the platform </a>

Finally, we can commit the first project version to the platform. 

In [ ]:
project.commit("Fixes data consistency issues (train-val leakage). Adds a full model")

In [ ]:
project.status()

In [ ]:
project.push()